<a href="https://colab.research.google.com/github/Vaycold/tensorflow_tutorial/blob/main/%234.TF%20Hub%EB%A1%9C%20%ED%85%8D%EC%8A%A4%ED%8A%B8%20%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("GPU", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가능")

GPU 사용 불가능


In [4]:
# Split train_data, test_into 6:4
# train_data : 15000, validation : 10000, test_data : 25000

train_data, validaition_data, test_data = tfds.load(
    name = 'imdb_reviews',
    split = ('train[:60%]', 'train[60%:]', 'test'), 
    as_supervised = True
)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6FLD38/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6FLD38/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6FLD38/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
train_examples_batch, train_label_batch = next(iter(train_data.batch(10))) # 처음 10개 sample 출력
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
train_label_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## Modeling

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding,
                           input_shape = [],
                           dtype = tf.string,
                           trainable = True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

model.summary(
    
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [11]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs = 20,
                    validation_data = validaition_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 4s 95ms/step - loss: 0.8189 - accuracy: 0.4817 - val_loss: 0.7365 - val_accuracy: 0.5252
Epoch 2/20
30/30 [==============================] - 3s 89ms/step - loss: 0.6995 - accuracy: 0.5582 - val_loss: 0.6717 - val_accuracy: 0.5811
Epoch 3/20
30/30 [==============================] - 3s 89ms/step - loss: 0.6440 - accuracy: 0.6085 - val_loss: 0.6252 - val_accuracy: 0.6290
Epoch 4/20
30/30 [==============================] - 3s 90ms/step - loss: 0.5983 - accuracy: 0.6527 - val_loss: 0.5854 - val_accuracy: 0.6574
Epoch 5/20
30/30 [==============================] - 3s 91ms/step - loss: 0.5583 - accuracy: 0.6881 - val_loss: 0.5507 - val_accuracy: 0.7094
Epoch 6/20
30/30 [==============================] - 3s 91ms/step - loss: 0.5206 - accuracy: 0.7287 - val_loss: 0.5167 - val_accuracy: 0.7277
Epoch 7/20
30/30 [==============================] - 3s 91ms/step - loss: 0.4844 - accuracy: 0.7563 - val_loss: 0.4868 - val_accuracy: 0.7521
Epoch 8/20
30

In [12]:
# Evaluation Model
results = model.evaluate(test_data.batch(512), verbose =2)
for name, value in zip(model.metrics_names, results) :
    print('%s : %.3f' % (name,value))

49/49 - 2s - loss: 0.3232 - accuracy: 0.8570
loss : 0.323
accuracy : 0.857
